    evolutionary pseudocode
    
    BEGIN
    INPUT cities, distance_matrix
    route = cities in order
    best_route = route
    best_distance = distance(route)
    stagnations = 0

    WHILE stagnations < 3
        improved = false

        FOR 100 times
            new_route = swap two cities in route
            IF distance(new_route) < best_distance THEN
                best_route = new_route
                best_distance = distance(new_route)
                route = new_route
                improved = true
            ENDIF
        ENDFOR

        IF improved = false THEN
            stagnations = stagnations + 1
            route = mutate(route) 5 times
        ENDIF
    ENDWHILE

    PRINT best_route, best_distance
    END

In [299]:
import random

def read_in(name_file, distance_file):
    '''
    Reads in the city names and distance matrix data.

    :param name_file: file containing city names
    :param distance_file: file containing a square distance matrix

    :return distance_result: 2D list of distances between cities
    :return city_names: list of city names
    '''

    with open(name_file, 'r') as f:
        city_names = [line.strip() for line in f.readlines() if line.strip()]

    distance_result = []
    with open(distance_file, 'r') as f:
        for line in f:
            distance_result.append([float(x) for x in line.split()])

    return distance_result, city_names

def compute_distance(route, distance_matrix, city_names):
        '''
        Calculates the total distance for a given route.

        :param route: list of city names representing a route
        :param distance_matrix: 2D list of distances between cities
        :param city_names: list of city names

        :return total: total distance of the given route
        '''

        total = 0
        for i in range(len(route) - 1):
            total += distance_matrix[city_names.index(route[i])][city_names.index(route[i + 1])]
        total += distance_matrix[city_names.index(route[-1])][city_names.index(route[0])]
        return total

In [300]:
def tsp_evolution(name_file, distance_file):
    '''
    Solves the Travelling Salesperson problem using an evolutionary algorithm.

    :param name_file: file containing city names
    :param distance_file: file containing a square distance matrix

    :return best_route: the best route found
    :return best_distance: the total distance of best_route
    '''

    distance_matrix, citiesLst = read_in(name_file, distance_file)
    best_parents = []
    stagnations = 0

    # INITIALIZE
    route = citiesLst[:]
    best_distance = compute_distance(route, distance_matrix, citiesLst)
    best_route = route.copy()

    while stagnations < 3:
        improved = False
        
        # MUTATE & SELECT
        for offspring_num in range(100):
            offspring = evo_mutate(route)
            offspring_distance = compute_distance(offspring, distance_matrix, citiesLst)

            if offspring_distance < best_distance:
                best_distance = offspring_distance
                best_route = offspring
                route = offspring
                improved = True

        if not improved:
            stagnations += 1
            best_parents.append(best_route.copy())
            route = stagnation(best_route)
    
    return f'{best_route}\n{best_distance}'

def evo_mutate(route):
    '''
    This function handles every mutation for the evolutionary algorithm.
    Returns a slightly different solution.
    '''

    random_city1 = random.randint(0, len(route) - 1)
    random_city2 = random.randint(0, len(route) - 1)
            
    while random_city1 == random_city2:
        random_city2 = random.randint(0, len(route) - 1)

    temp = route[random_city1]
    route[random_city1] = route[random_city2]
    route[random_city2] = temp
    
    return route

def stagnation(route):
    '''
    This function handles all stagnations that occur until the termination limit is reached.
    Mutates the given route 5 times.
    '''

    for i in range(5):
        route = evo_mutate(route)
    return route

In [301]:
print(tsp_evolution('thirty_cities_names.txt','thirty_cities_dist.txt'))

['Cairo', 'Guam', 'Mexico City', 'Azores', 'Sydney', 'Baghdad', 'Tokyo', 'Buenos Aires', 'Berlin', 'Rio de Janeiro', 'Panama City', 'Moscow', 'New Orleans', 'London', 'Chicago', 'Montreal', 'Manila', 'Bombay', 'Rome', 'Istanbul', 'Capetown', 'Seattle', 'Santiago', 'Melbourne', 'Juneau', 'Shanghai', 'Honolulu', 'New York', 'Paris', 'San Francisco']
1344.0


# Greedy Pseudocode 

Choose a starting point within the dataset and make it the current number.

for each City

    pick the closest city to you current city 

    remove the city from a valid list

    continue the process until you have selected all of the cities.

add all the numbers to calculate the total distance.

In [302]:
def compute_distance(route, distance_matrix):
    """Compute total round-trip distance of a given route."""
    total = 0
    for i in range(len(route) - 1):
        total += distance_matrix[route[i]][route[i + 1]]
    total += distance_matrix[route[-1]][route[0]]  # return to start
    return total
def distance_from_current(city, current_city, distance_matrix):
    """Return distance between the current city and a given city."""
    return distance_matrix[current_city][city]
def tsp_greedy(name_file, distance_file):
    """
    Solves the TSP using a single greedy run from a user-chosen starting city.
    Parameters:
        name_file: path to text file containing city names
        distance_file: path to text file containing distance matrix
    Returns:
        named_route: list of city names in the greedy route order
        total_distance: total round-trip distance of that route
    """
    # Opens and reads input files
    with open(name_file, 'r') as f:
        city_names = [line.strip() for line in f.readlines() if line.strip()]
    distance_matrix = []
    with open(distance_file, 'r') as f:
        for line in f:
            row = [float(x) for x in line.split()]
            distance_matrix.append(row)
    n = len(city_names)
    # Choose a starting city
    while True:
        try:
            start_city = int(input(f"Enter the number of your starting city (0 to {n - 1}): "))
            if 0 <= start_city < n:
                break
            else:
                print(f"Invalid input. Please enter a number between 0 and {n - 1}.")
        except ValueError:
            print("Invalid input. Please enter an integer.")
    # Initialize
    unvisited = list(range(n))
    route = [start_city]
    unvisited.remove(start_city)
    current_city = start_city
    # Build route
    while unvisited:
        next_city = min(unvisited, key=lambda city: distance_from_current(city, current_city, distance_matrix))
        route.append(next_city)
        unvisited.remove(next_city)
        current_city = next_city
    total_distance = compute_distance(route, distance_matrix)
    named_route = [city_names[i] for i in route]
    # Print results
    print("Starting City:", city_names[start_city])
    print("Greedy Route:", " -> ".join(named_route) + f" -> {named_route[0]}")
    print(f"Total Distance: {total_distance}")

In [303]:
tsp_greedy('thirty_cities_names.txt', 'thirty_cities_dist.txt')

Starting City: Azores
Greedy Route: Azores -> London -> Paris -> Berlin -> Rome -> Istanbul -> Cairo -> Baghdad -> Moscow -> Bombay -> Shanghai -> Tokyo -> Guam -> Manila -> Melbourne -> Sydney -> Honolulu -> San Francisco -> Seattle -> Juneau -> Chicago -> New York -> Montreal -> New Orleans -> Mexico City -> Panama City -> Santiago -> Buenos Aires -> Rio de Janeiro -> Capetown -> Azores
Total Distance: 527.0
